In [1]:
import pandas as pd
import sys

In [2]:
nccdc = pd.read_csv('NorthCarolinaVulnerability/NorthCarolina.csv')
nccdc.head()

,FID,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,0,37,NORTH CAROLINA,NC,37019,Brunswick,37019990100,"Census Tract 9901, Brunswick County, North Car...",0.000000,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,12.0,-999.0,-999.0,0.0
1,1,37,NORTH CAROLINA,NC,37031,Carteret,37031980100,"Census Tract 9801, Carteret County, North Caro...",21.918898,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,12.0,-999.0,-999.0,0.0
2,2,37,NORTH CAROLINA,NC,37031,Carteret,37031990100,"Census Tract 9901, Carteret County, North Caro...",0.000000,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,12.0,-999.0,-999.0,0.0
3,3,37,NORTH CAROLINA,NC,37051,Cumberland,37051980100,"Census Tract 9801, Cumberland County, North Ca...",35.598407,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,12.0,-999.0,-999.0,5250.0
4,4,37,NORTH CAROLINA,NC,37053,Currituck,37053990100,"Census Tract 9901, Currituck County, North Car...",0.000000,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,12.0,-999.0,-999.0,0.0


In [3]:
cumbercdc = nccdc.loc[nccdc.STCNTY == 37051]
cumbercdc.to_csv('NorthCarolinaVulnerability/Cumberland.csv',index=False)
cumbercdc.head()

,FID,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
3,3,37,NORTH CAROLINA,NC,37051,Cumberland,37051980100,"Census Tract 9801, Cumberland County, North Ca...",35.598407,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,12.0,-999.0,-999.0,5250.0
382,382,37,NORTH CAROLINA,NC,37051,Cumberland,37051000200,"Census Tract 2, Cumberland County, North Carolina",2.614907,2148.0,...,0.0,1.0,0.0,1.0,7.0,326.0,101.0,15.2,4.6,3723.0
383,383,37,NORTH CAROLINA,NC,37051,Cumberland,37051000500,"Census Tract 5, Cumberland County, North Carolina",1.698971,2231.0,...,0.0,1.0,0.0,1.0,5.0,354.0,69.0,16.1,2.9,3063.0
384,384,37,NORTH CAROLINA,NC,37051,Cumberland,37051000600,"Census Tract 6, Cumberland County, North Carolina",1.958944,5573.0,...,0.0,0.0,0.0,0.0,0.0,666.0,314.0,12.4,5.5,6247.0
385,385,37,NORTH CAROLINA,NC,37051,Cumberland,37051000701,"Census Tract 7.01, Cumberland County, North Ca...",2.456289,5539.0,...,0.0,0.0,0.0,0.0,0.0,323.0,128.0,5.9,2.3,7774.0


In [4]:
f = open("NorthCarolinaVulnerability/Cumberland.sql","w")
f.write("\copy cumberland_cdc(")

for col in cumbercdc.columns:
    if col == "E_DAYPOP":
        f.write(col+") ")
    else:
        f.write(col+",")

f.write("FROM 'Cumberland.csv' WITH DELIMITER ',' CSV HEADER;\n")
f.write("COMMIT;")
f.close()

In [5]:
f = open("NorthCarolinaVulnerability/SchemaCumberland.sql","w")
f.write("SET CLIENT_ENCODING TO UTF8;\n")
f.write("SET STANDARD_CONFORMING_STRINGS TO ON;\n")
f.write("BEGIN;\n")
f.write("CREATE TABLE cumberland_cdc (gid serial,\n")

for col in cumbercdc.columns:
    dt = cumbercdc[col].dtype
    if dt == "int64":
        dtype = "NUMERIC"
    elif dt == "object":
        dtype = "VARCHAR(60)"
    elif dt == "float64":
        dtype = "DOUBLE PRECISION"
    else:
        sys.exit('Incorrect dtype')
        
    if col == "E_DAYPOP":
        f.write("    "+col+" "+dtype+");\n")
    else:
        f.write("    "+col+" "+dtype+",\n")

f.write("ALTER TABLE cumberland_cdc ADD PRIMARY KEY (gid);\n")
f.write("COMMIT;")
f.close()